<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Reddit API:
1. Go to https://www.reddit.com/prefs/apps
2. Click "create application"
3. Name: My Sentiment Example
4. Choose script
5. Leave *description* and *about url* blank
6. Add http://www.example.com/unused/redirect/uri to *redirect uri*

## Imports

In [19]:
import praw
import openai

In [91]:
reddit = praw.Reddit(client_id="TODO", # the App ID retreived from the reddit application
                     client_secret="TODO", # the secret retreived from the reddit application
                     user_agent="sentiment analysis test (by u/YourUserName)"  # add your reddit name
                    )

We can now already use our reddit instance to obtain public information from reddit!

In [5]:
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)


test
While I'm all for testing out bots in this subreddit, these accounts have been making hundreds of posts per week. For many months. I follow this subreddit because it's interesting to see what people are using it for. These accounts flood my homepage on a daily basis.
-
asd
cltv test
cool
Sample File Based Image Part 2
Sample File Based Image
It didn't seem like trouble at first.
kgjkjg


Of course we want to access hot subreddits like r/wallstreetbets or r/stocks

In [12]:
subreddit_stocks = reddit.subreddit("stocks")

In [16]:
print(subreddit_stocks.display_name)
print(subreddit_stocks.title)
print(subreddit_stocks.accounts_active)

stocks
Stocks - Investing and trading for all
5871


Let's say we want to access the first 20 hot posts:

In [17]:
for post in subreddit_stocks.hot(limit=20):
    print(post.title)

Rate My Portfolio - r/Stocks Quarterly Thread December 2022
r/Stocks Daily Discussion & Technicals Tuesday - Jan 31, 2023
PayPal to lay off 2,000 employees in coming weeks, about 7% of workforce
AMD Earnings - beats on sales and profit but warns of a 10% revenue decline in Q1
Intel cutting employee compensation to preserve cash as it struggles with a rapid drop in revenue and earnings
Snap shares plunge on weak revenue as company again declines to provide a forecast
Jim Cramer says we’re in a bull market, so buy on the dip
Selling everything before fed speech, then buying back cheaper?
J&J denied Texas two step strategy
SoFi aims to outgrow the Fintech Market.
GM smashes expectations and guides toward a strong 2023, despite margin squeeze
10 lessons from the investing book “Margin of Safety” by Seth Klarman:
Apple’s expected to post its first revenue decline since 2019 on Thursday
Meta technology chief Bosworth implies company has lost focus
Article: Justice Department seeks Tesla auto

We can now use GPT to extract stock tickers from all of these posts and tell us whether the sentiment for the mentioned assets is positive, neutral or negative:

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [18]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!

In [20]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Prompt Generation
First, we need to create a formatted string of all titles

In [24]:
getattr(subreddit_stocks, "hot")

<bound method BaseListingMixin.hot of Subreddit(display_name='stocks')>

In [83]:
def get_titles(subreddit="stocks", sub_instance="hot", limit=10, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles = ""
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):

        if c < skip_first:
            continue
        
        c+=(1-skip_first)

        titles += str(c) + ". "
        titles += post.title
        titles += "\n"
    return titles

In [84]:
titles = get_titles(subreddit="stocks", limit=12)

In [85]:
print(titles)

1. PayPal to lay off 2,000 employees in coming weeks, about 7% of workforce
2. AMD Earnings - beats on sales and profit but warns of a 10% revenue decline in Q1
3. Intel cutting employee compensation to preserve cash as it struggles with a rapid drop in revenue and earnings
4. Snap shares plunge on weak revenue as company again declines to provide a forecast
5. Jim Cramer says we’re in a bull market, so buy on the dip
6. Selling everything before fed speech, then buying back cheaper?
7. J&J denied Texas two step strategy
8. SoFi aims to outgrow the Fintech Market.
9. GM smashes expectations and guides toward a strong 2023, despite margin squeeze
10. 10 lessons from the investing book “Margin of Safety” by Seth Klarman:



In [86]:
def create_prompt(post_titles):

    task = "Return the stock ticker or company in the following headings and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + post_titles


In [87]:
prompt = create_prompt(titles)

In [88]:
print(prompt)

Return the stock ticker or company in the following headings and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

1. PayPal to lay off 2,000 employees in coming weeks, about 7% of workforce
2. AMD Earnings - beats on sales and profit but warns of a 10% revenue decline in Q1
3. Intel cutting employee compensation to preserve cash as it struggles with a rapid drop in revenue and earnings
4. Snap shares plunge on weak revenue as company again declines to provide a forecast
5. Jim Cramer says we’re in a bull market, so buy on the dip
6. Selling everything before fed speech, then buying back cheaper?
7. J&J denied Texas two step strategy
8. SoFi aims to outgrow the Fintech Market.
9. GM smashes expectations and guides toward a strong 2023, despite margin squeeze
10. 10 lessons from the investing book “Margin of Safety” by Seth Klarman:



In [89]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=prompt,
                                            max_tokens=256,
                                            temperature=0,
                                            top_p=1.0,
                                            frequency_penalty=0.0,
                                            presence_penalty=0.0)


In [90]:
print(response["choices"][0]["text"])


1. PayPal - No sentiment
2. AMD - Negative
3. Intel - Negative
4. Snap - Negative
5. No company mentioned - No sentiment
6. No company mentioned - No sentiment
7. J&J - No sentiment
8. SoFi - Positive
9. GM - Positive
10. No company mentioned - No sentiment
